# Recommender system

# WARNING

## DO NOT 'RUN ALL' the cells, some of them take a really long time, read the content before running anything



## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from sklearn.model_selection import train_test_split

## Data loading

NOTE : resources folder could not be included on github due to size. It also stores cached NMF results.

In [2]:
train_file_path = 'resources/lab2_train.csv'
test_file_path = 'resources/lab2_test.csv'

train_data = pd.read_csv(train_file_path, delimiter=',')

train_bool = train_data.copy()
train_bool['is_match'] = train_bool['is_match'].astype('bool')

test_data = pd.read_csv(test_file_path, delimiter=',')

print("Train data info")
print(train_data.describe())
print("\nTest data info")
print(test_data.describe())

Train data info
       user_from_id    user_to_id
count  76392.000000  76392.000000
mean    1135.738297   1817.058174
std      818.555235   1059.191942
min        1.000000      0.000000
25%      443.000000    887.000000
50%      988.000000   1807.000000
75%     1707.000000   2733.250000
max     3716.000000   3624.000000

Test data info
       user_from_id    user_to_id
count  16203.000000  16203.000000
mean    1389.043264   1777.779053
std      952.172409   1053.127740
min        0.000000      0.000000
25%      583.000000    835.000000
50%     1219.000000   1836.000000
75%     2087.500000   2676.000000
max     3719.000000   3624.000000


## Familiarization

We will start by examing the distribution of likes and matches

In [7]:
train_n = train_data.shape[0]
train_likes = train_data[train_data['is_like'] == True].shape[0]
train_matches = train_data[train_data['is_match'] == True].shape[0]
train_nulls = train_data[train_data['is_match'].isnull() == True].shape[0]
print("Train data")
print(f'Like: {train_likes / train_n * 100}%')
print(f"Match: {train_matches / train_n * 100}%  Nulls: {train_nulls / train_n * 100}%")


bool_n = train_bool.shape[0]
bool_matches = train_bool[train_bool['is_match'] == True].shape[0]
print("\nTrain data bool (like stays the same)")
print(f"Match percentage: {bool_matches / bool_n * 100}%")


Train data
Like: 16.54230809508849%
Match: 0.5432506021572939%  Nulls: 1.0668656403811918%

Train data bool (like stays the same)
Match percentage: 1.6101162425384858%


We can see that is_like and is_match are both sparse, with only 0.5% of entries being a match.

Null values in is_match probably represent someone who has been liked, but not "responded" with a like or rejection of their own. These values can be removed while training.

We move on to some vizualization

In [ ]:
df = train_data.copy()

In [ ]:
likes_given = df['user_from_id'].value_counts()
likes_received = df['user_to_id'].value_counts()


# Plotting
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
sns.histplot(likes_given, kde=False, color='blue', label='Likes Given')
sns.histplot(likes_received, kde=False, color='orange', label='Likes Received')
plt.title('Distribution of Likes Given and Received')
plt.xlabel('likes given / received')
plt.ylabel('nr of users')
plt.legend()

plt.subplot(1, 2, 2)
sns.histplot(likes_given[likes_given > 75], kde=False, color='blue', label='Likes Given')
sns.histplot(likes_received[likes_received > 75], kde=False, color='orange', label='Likes Received')
plt.title('Distribution of likes Given and Received - top preformers')
plt.xlabel('likes given / received')
plt.ylabel('nr of users')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Create a directed graph
G = nx.DiGraph()

# Calculate the number of likes received by each user
likes_received = train_data[train_data['is_like']].groupby('user_to_id').size()

# Determine the top 10% of users based on likes received
top_percentage = 0.1
top_users_count = int(len(likes_received) * top_percentage)
top_user_ids = likes_received.nlargest(top_users_count).index

# Add all user nodes to the graph
G.add_nodes_from(train_data['user_from_id'])
G.add_nodes_from(train_data['user_to_id'])

# Add edges from top users
for _, row in train_data.iterrows():
    if row['user_to_id'] in top_user_ids or row['user_from_id'] in top_user_ids:
        G.add_edge(row['user_from_id'], row['user_to_id'], is_like=row['is_like'], is_match=row['is_match'])

# Define node colors based on whether the user is a top user or not
node_color = ['yellow' if node in top_user_ids else 'skyblue' for node in G]

# Define edge colors based on whether it's a like or a match
edge_color = ['green' if data['is_match'] else 'gray' for _, _, data in G.edges(data=True)]

# Position nodes using a spring layout
pos = nx.spring_layout(G, k=0.2, iterations=20)

# Draw the nodes
nx.draw_networkx_nodes(G, pos, node_color=node_color, alpha=0.7)

# Draw the edges
nx.draw_networkx_edges(G, pos, edge_color=edge_color, alpha=0.5)

# Set plot title and show graph
plt.title("User Interactions: Likes and Matches")
plt.axis('off')
plt.show()

In [ ]:
## Top users likes

# init df
df = train_data.copy()

# Create a directed graph
G = nx.DiGraph()

# find top 10% of users (most received likes)
likes_received = df[df['is_like']].groupby('user_to_id').size()
top_users = likes_received.sort_values(ascending=False)
top_percentage = 0.1
top_users_count = int(len(top_users) * top_percentage)

top_user_ids = top_users.head(top_users_count).index
top_users_df = df[df['user_to_id'].isin(top_user_ids)]


# Add nodes
nodes = pd.concat([df['user_from_id'], df['user_to_id']])
G.add_nodes_from(nodes.unique())

# add edges for top users
for _, row in df.iterrows():
    if row['user_from_id'] in top_user_ids:
        G.add_edge(row['user_from_id'], row['user_to_id'], is_like=row['is_like'], is_match=row['is_match'])

# Define node colors based on whether the user is a top user or not
node_color = ['yellow' if node in top_user_ids else 'skyblue' for node in G]

# Define edge colors based on whether it's a like or a match
edge_color = ['green' if data['is_match'] else 'gray' for _, _, data in G.edges(data=True)]

# set figure and pos
plt.figure(figsize=(20, 11))
pos = nx.spring_layout(G, k=0.2, iterations=20)

# Draw nodes
node_degree = dict(G.degree)
nx.draw_networkx_nodes(G, pos, node_size=[(1 + v) for v in node_degree.values()], node_color=node_color, alpha=0.7)

# Draw edges
nx.draw_networkx_edges(G, pos, edge_color=edge_color, alpha=0.5)

# Set plot title and show graph
plt.title("User Interactions: Likes and Matches")
plt.axis('off')
plt.show()

In [ ]:
G = nx.DiGraph()

# Add nodes and edges with weights for likes
for _, row in train_data.iterrows():
    G.add_node(row['user_from_id'])
    G.add_node(row['user_to_id'])
    weight = 1 if row['is_like'] else 0
    match_weight = 3 if row['is_match'] else 0
    G.add_edge(row['user_from_id'], row['user_to_id'], weight=weight, match_weight=match_weight)

# Use a force-directed layout to spread nodes out
plt.figure(figsize=(20, 11))
pos = nx.spring_layout(G, k=0.3, iterations=20)

# Draw nodes
node_degree = dict(G.degree)
nx.draw_networkx_nodes(G, pos, node_size=[v * 2 for v in node_degree.values()], node_color='skyblue', alpha=0.7)

# Draw likes edges
likes_edges = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] == 1]
nx.draw_networkx_edges(G, pos, edgelist=likes_edges, edge_color='gray', alpha=0.05)

# # Draw matches edges
# matches_edges = [(u, v) for (u, v, d) in G.edges(data=True) if d['match_weight'] == 3]
# nx.draw_networkx_edges(G, pos, edgelist=matches_edges, edge_color='green', alpha=0.5)

# Set plot title and show graph
plt.title("User Interactions: Likes and Matches")
plt.axis('off')
plt.show()

## Non negative matrix factorization

In [9]:
nnm_train = train_data.copy()
nnm_test = test_data.copy()

#### Methods

In [4]:
def nmf_loss(V, W, H):
    # Create mask 
    mask = V != 0

    # find error
    reconstruction = np.dot(W, H)
    error = V - reconstruction

    # Apply mask and return frobenius norm.
    masked_error = np.where(mask, error, 0)
    return np.linalg.norm(masked_error, 'fro')

In [5]:
def nmf(X: pd.DataFrame, n_components: int, max_iter: int=1000, tol: float=1e-3, debug: bool=False):
  """
  Decomposes the original sparse matrix X into two matrices W and H. 
  """
  # Initialize W and H with random non-negative values
  W = np.random.rand(X.shape[0], n_components)
  H = np.random.rand(n_components, X.shape[1])

  # Set constants
  epsilon = 1e-9;
  V = X.to_numpy() if isinstance(X, pd.DataFrame) else X;

  # Construct intial error
  error = nmf_loss(V, W, H);
  new_error = 0;

  iterations = 0;
  while (error - new_error) > tol and iterations <= max_iter:
    if (debug and iterations%10 == 0):
      print(f"{iterations}/{max_iter}")
    W_update = np.dot(V, H.T) / (np.dot(np.dot(W, H), H.T) + epsilon)
    W *= W_update 

    H_update = np.dot(W.T, V) / (np.dot(np.dot(W.T, W), H) + epsilon)
    H *= H_update

    new_error = nmf_loss(V, W, H)
    iterations+=1;


  return W, H

In [6]:
def nmf_optimized(X: pd.DataFrame, n_components: int, max_iter: int = 1000, tol: float = 1e-3, debug: bool = False):
    """
    Decomposes the original sparse matrix X into two matrices W and H.
    """

    # Set constants
    epsilon = 1e-9
    V = X.to_numpy() if isinstance(X, pd.DataFrame) else X

    # Initialize W and H with random non-negative values, scaled to max of X
    max = V.flatten().max();
    W = np.random.rand(X.shape[0], n_components) * max;
    H = np.random.rand(n_components, X.shape[1]) * max;

    # Construct initial error
    error = nmf_loss(V, W, H)
    new_error = 0

    iterations = 0
    while iterations < max_iter:
        if debug and iterations % 10 == 0:
            print(f"{iterations}/{max_iter}")

        # Update W
        W_update = np.dot(V, H.T) / (np.dot(np.dot(W, H), H.T) + epsilon)
        W *= W_update
        new_error = nmf_loss(V, W, H)
        
        # Check for convergence after updating W
        if abs(error - new_error) < tol:
            break
        error = new_error

        # Update H
        H_update = np.dot(W.T, V) / (np.dot(np.dot(W.T, W), H) + epsilon)
        H *= H_update
        new_error = nmf_loss(V, W, H)

        # Check for convergence after updating H
        if abs(error - new_error) < tol:
            break
        error = new_error

        iterations += 1

    return W, H


In [7]:
def nmf_validate(prediction_matrix, validation_set, threshhold):
    FP = 0
    FN = 0
    TP = 0
    TN = 0

    for _, row in validation_set.iterrows():
        pred_value = prediction_matrix.iloc[row['user_from_id'], row['user_to_id']]
        pred = 2 if pred_value > threshhold else 1

        if(row['is_like'] and pred == 2):
            TP+=1
        if(row['is_like'] and pred == 1):
            FN+=1
        if(not row['is_like'] and pred == 2):
            FP+=1
        if(not row['is_like'] and pred == 1):
            TN+=1

    
    
    accuracy = (TP + TN) / (TP + FP + FN + TN)
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return {"FP": FP, "FN": FN, "TP": TP, "TN": TN, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1_score}


#### Data cleaning and setup

We start by checking if we can remove any outliers, and what an outlier represents in the context of the dating app

In [10]:
# create user profiles with like stats
user_profiles = nnm_train.groupby('user_to_id')['is_like'].agg(like_count='sum', total_count='count')
user_profiles['like_count'] = user_profiles['like_count'].astype(int)  
user_profiles['rejection_count'] = user_profiles['total_count'] - user_profiles['like_count']
user_profiles['like_ratio'] = user_profiles['like_count'] / user_profiles['total_count']

# Define the percentile thresholds for outliers (manually adjusted)
lower_thresh = user_profiles['like_ratio'].quantile(0.05)
upper_thresh = user_profiles['like_ratio'].quantile(0.98)

# Identify users who are outliers based on the like ratio
outliers = user_profiles[(user_profiles['like_ratio'] < lower_thresh) | 
                         (user_profiles['like_ratio'] > upper_thresh)]

print(f'upper:{upper_thresh}  lower:{lower_thresh}')
print(f'outliers removed {outliers.shape[0]} out {nnm_train.shape[0]}')
print(user_profiles.describe())
print(outliers.sample(10))

upper:0.7142857142857143  lower:0.0
outliers removed 57 out 76392
        like_count  total_count  rejection_count   like_ratio
count  3040.000000  3040.000000      3040.000000  3040.000000
mean      4.156908    25.128947        20.972039     0.154255
std       7.176296    28.938301        24.850343     0.208570
min       0.000000     1.000000         0.000000     0.000000
25%       0.000000     6.000000         5.000000     0.000000
50%       1.000000    16.000000        13.000000     0.044281
75%       5.000000    33.250000        27.000000     0.270270
max      57.000000   362.000000       350.000000     1.000000
            like_count  total_count  rejection_count  like_ratio
user_to_id                                                      
3130                 3            3                0    1.000000
645                  1            1                0    1.000000
2712                 1            1                0    1.000000
1973                 2            2                

Based on our user profiles, we could potentially remove outliers using like count, rejection count, total count, or like ratio.

- **total count**
Outliers based on total count are just the most and least active users. It might be good to remove the least active ones
- **like count**
Same problem as total count while also penalizing "desirable" users
- **rejection count**
Same problem as total count while also penalizing "undesirable" users
- **like ratio**
Penalizes the most and least "desirable" users, also removes users with low activity (as their ratio is liklier to be high or low)

We think removing outliers would degrade the quality of the data, with the possible exception of removing barely active users.

Thus we construct our user-user matrix with the full training set, with na values in is_match dropped (as they represent only 1% of the dataset, and cannot be easily represented). We will not normalize as it is not applicable in our case, and represent likes as 20, rejections as 10, and no interaction as 0. The nmf_optimized subroutine has been adjusted for this, and we belive the larger numbers will allow likes to be more easily distingused.

In [15]:
max_user_id = max(nnm_train['user_from_id'].max(), nnm_train['user_to_id'].max())
interaction_matrix = np.zeros((max_user_id + 1, max_user_id + 1))


# Populate the matrix
for _, row in nnm_train.iterrows():
    value = 20 if row['is_like'] else 10
    interaction_matrix[row['user_from_id'], row['user_to_id']] = value

# Split the data
train_df, val_df = train_test_split(nnm_train, test_size=0.1)

# Mask interactions for validation set
for _, row in val_df.iterrows():
    interaction_matrix[row['user_from_id'], row['user_to_id']] = 0

Now we apply our nmf subroutine to get our prediction matrix. We use k = 64 initially but will optimize this later

In [16]:
W, H = nmf_optimized(interaction_matrix, 64, tol=1e-4, debug=True, max_iter=666)
prediction_matrix = pd.DataFrame(np.dot(W, H))
print(prediction_matrix.shape)

print(prediction_matrix.to_numpy().flatten().mean())
print(prediction_matrix.to_numpy().flatten().std())
print(np.quantile(prediction_matrix.to_numpy().flatten(), 0.299))



0/666
10/666
20/666
30/666
40/666


KeyboardInterrupt: 

In [36]:
flat_preds = prediction_matrix.to_numpy().flatten()

thresh2 = flat_preds.mean() + flat_preds.std() * 2;
thresh3 = flat_preds.mean() + flat_preds.std() * 3;
thresh4 = flat_preds.mean() + flat_preds.std() * 4;

print(np.quantile(flat_preds, 0.99))


print(f"Thresh: {thresh2} {nmf_validate(prediction_matrix=prediction_matrix, validation_set=val_df, threshhold=thresh2)}")
print(f"Thresh: {thresh3} {nmf_validate(prediction_matrix=prediction_matrix, validation_set=val_df, threshhold=thresh3)}")
print(f"Thresh: {thresh4} {nmf_validate(prediction_matrix=prediction_matrix, validation_set=val_df, threshhold=thresh4)}")


1.6020704361211726
Thresh: 0.9372662073337882 {'FP': 4004, 'FN': 468, 'TP': 810, 'TN': 2358, 'Accuracy': 0.41465968586387436, 'Precision': 0.1682592438720399, 'Recall': 0.6338028169014085, 'F1 Score': 0.26592252133946165}
Thresh: 1.371535794910176 {'FP': 3263, 'FN': 594, 'TP': 684, 'TN': 3099, 'Accuracy': 0.4951570680628272, 'Precision': 0.17329617430960223, 'Recall': 0.5352112676056338, 'F1 Score': 0.26181818181818184}
Thresh: 1.805805382486564 {'FP': 2630, 'FN': 690, 'TP': 588, 'TN': 3732, 'Accuracy': 0.5654450261780105, 'Precision': 0.18272218769422002, 'Recall': 0.460093896713615, 'F1 Score': 0.2615658362989324}


Great! now that we now everything works, we need to find the optimal k and threshold.

Since training takes a very long time, we will create seperate prediction matrices for differing numbers of components, and initially evaluate them using a statistical threshold 3std from the mean. We then loop through different quantile thresholds and statistical thresholds to find the optimal one. Afterwards, we can pick the best k / threshold combination based on its preformance (accuracy, precision, F1, Etc.)

Note that we will be saving the prediction matrices as to not have to recompute them later.

Additionally, we observe that due to the mostly sparse matrix (most users have no interaction), and the relative rareness of likes (around 15%), our threshold will likley have to be quite high. We will check quantiles ranging from 0.8 to 1 and thresholds up to 4 standard deviations from the mean.



In [11]:
def find_optimal_threshold(prediction_matrix:pd.DataFrame, validation_set, metric='F1 Score'):
    best_threshold = 0
    best_metric_value = 0
    metrics_history = []

    # Flatten the prediction matrix to apply quantiles
    flattened_predictions = prediction_matrix.to_numpy().flatten()

    # Generate quantiles starting from 0.8
    quantiles = np.linspace(0.8, 1, 100)  # Adjust number of quantiles as needed for time efficiency

    for q in quantiles:
        threshold = np.quantile(flattened_predictions, q)

        # Validate with the current threshold
        validation_results = nmf_validate(prediction_matrix, validation_set, threshold)

        # Determine the current metric value
        current_metric_value = validation_results[metric]
        if current_metric_value == 0:
            break
        # print(f"Quantile: {q}, Threshold: {threshold}, {metric}: {current_metric_value}")

        # Store the history of metrics for analysis
        metrics_history.append((threshold, current_metric_value))

        # Update the best threshold if the current metric is better
        if current_metric_value > best_metric_value:
            best_metric_value = current_metric_value
            best_threshold = threshold

    # Check thresholds based of standard deviation from mean
    for i in range(1, 5):
        threshold = flattened_predictions.mean() + flattened_predictions.std() * i

        validation_results = nmf_validate(prediction_matrix, validation_set, threshold)

        # Determine the current metric value
        current_metric_value = validation_results[metric]

        # Store the history of metrics for analysis
        metrics_history.append((threshold, current_metric_value))

        # Update the best threshold if the current metric is better
        if current_metric_value > best_metric_value:
            best_metric_value = current_metric_value
            best_threshold = threshold


    print(f"found optimal threshold {best_threshold} with F1 {best_metric_value}")
    return best_threshold, best_metric_value, metrics_history

In [12]:
def initial_validation(prediction_matrix: pd.DataFrame, val_df:pd.DataFrame):
    flat_preds = prediction_matrix.to_numpy().flatten()

    thresh = flat_preds.mean() + flat_preds.std() * 3

    results = nmf_validate(prediction_matrix, val_df, thresh)

    print(results)

    

In [25]:
# prediction matrix with 256 components
W, H = nmf_optimized(interaction_matrix, 256, tol=1e-2, debug=True, max_iter=666)
prediction_matrix_256 = pd.DataFrame(np.dot(W, H))

prediction_matrix_256.to_csv('./resources/256-cache.csv')

initial_validation(prediction_matrix_256, val_df)

0/666
10/666
20/666
30/666
{'FP': 2544, 'FN': 847, 'TP': 412, 'TN': 3837, 'Accuracy': 0.556151832460733, 'Precision': 0.13937753721244925, 'Recall': 0.3272438443208896, 'F1 Score': 0.19549228944246738}


In [50]:
# prediction matrix with 512 components
W, H = nmf_optimized(interaction_matrix, 512, tol=1e-2, debug=True, max_iter=666)
prediction_matrix_512 = pd.DataFrame(np.dot(W, H))

prediction_matrix_512.to_csv('./resources/512-cache.csv')

initial_validation(prediction_matrix_512, val_df)

0/666
10/666
20/666
30/666
40/666
50/666
60/666
70/666
80/666
90/666
100/666
110/666
120/666
130/666
140/666
150/666
{'FP': 1446, 'FN': 1079, 'TP': 199, 'TN': 4916, 'Accuracy': 0.6695026178010471, 'Precision': 0.1209726443768997, 'Recall': 0.15571205007824726, 'F1 Score': 0.13616147793362982}


In [51]:
# prediction matrix with 1024 components
W, H = nmf_optimized(interaction_matrix, 1024, tol=1e-2, debug=True, max_iter=666)
prediction_matrix_1024 = pd.DataFrame(np.dot(W, H))

prediction_matrix_1024.to_csv('./resources/1024-cache.csv')

initial_validation(prediction_matrix_1024, val_df)

0/666
10/666
20/666
30/666
40/666
50/666
60/666
70/666
{'FP': 834, 'FN': 1202, 'TP': 76, 'TN': 5528, 'Accuracy': 0.7335078534031414, 'Precision': 0.08351648351648351, 'Recall': 0.0594679186228482, 'F1 Score': 0.06946983546617916}


In [52]:
# prediction matrix with 2048 components
W, H = nmf_optimized(interaction_matrix, 2048, tol=1e-2, debug=True, max_iter=666)
prediction_matrix_2048 = pd.DataFrame(np.dot(W, H))

prediction_matrix_2048.to_csv('./resources/2048-cache.csv')

initial_validation(prediction_matrix_2048, val_df)

0/666
10/666
20/666
30/666
40/666
50/666
60/666
70/666
80/666
90/666
100/666
110/666
120/666
130/666
140/666
150/666
160/666
170/666
{'FP': 223, 'FN': 1267, 'TP': 11, 'TN': 6139, 'Accuracy': 0.8049738219895288, 'Precision': 0.04700854700854701, 'Recall': 0.008607198748043818, 'F1 Score': 0.01455026455026455}


In [57]:
prediction_matrix_256_read = pd.read_csv('./resources/256-cache.csv', delimiter=',', index_col=0)

print(prediction_matrix_256.shape)
print(prediction_matrix_256_read.shape)

(3717, 3717)
(3717, 3717)


In [59]:
# Reload prediction matrices

prediction_matrix_256 = pd.read_csv('./resources/256-cache.csv', delimiter=',', index_col=0)
prediction_matrix_512 = pd.read_csv('./resources/512-cache.csv', delimiter=',', index_col=0)
prediction_matrix_1024 = pd.read_csv('./resources/1024-cache.csv', delimiter=',', index_col=0)
prediction_matrix_2048 = pd.read_csv('./resources/2048-cache.csv', delimiter=',', index_col=0)

In [61]:
# Find optimal thresholds

stats_256 = find_optimal_threshold(prediction_matrix_256, val_df)
stats_512 = find_optimal_threshold(prediction_matrix_512, val_df)
stats_1024 = find_optimal_threshold(prediction_matrix_1024, val_df)
stats_2048 = find_optimal_threshold(prediction_matrix_2048, val_df)

found optimal threshold 0.0006438129481657227 with F1 0.27842914058053503
found optimal threshold 7.926719150537305 with F1 0.3966745843230404
found optimal threshold 9.871739404216756 with F1 0.4729276473406803
found optimal threshold 9.997559264606931 with F1 0.520923520923521


Right, it seems that the initial validation function was kind of useless, as the threshold worked better for lower component numbers. Going off the optimal thresholds however, we see a consitent improvement in f1 score as we add more components, going past 2048 is unfeasible though.

Running nmf with 2048 components took around 1.5 hours on my machine, which is simply too long. Thus, i decided to save all our prediction matrices and will be using 1024 components going forwards. This provides the best balance of performance and runtime in our opinion.

Before making our final predictions we will try running nmf on a normalized and min-max scaled interaction matrix. The normalized one will use 1024 components, but since it took so long to run, the min-max scaled version will have to use 512 and a lower tolerence.

In [24]:
# Trying it out with normalized interaction-matrix
interaction_matrix_norm = (interaction_matrix - interaction_matrix.mean()) / interaction_matrix.std()
W, H = nmf_optimized(interaction_matrix_norm, 1024, tol=1e-2, debug=True)
prediction_matrix_norm = pd.DataFrame(np.dot(W, H))

prediction_matrix_norm.to_csv('./resources/norm-cache.csv')

0/1000
10/1000
20/1000
30/1000
40/1000
50/1000
60/1000
70/1000
80/1000
90/1000
100/1000
110/1000
120/1000
130/1000
140/1000
150/1000
160/1000
170/1000
180/1000
190/1000
200/1000
210/1000
220/1000
230/1000
240/1000
250/1000
260/1000
270/1000
280/1000
290/1000
300/1000
310/1000
320/1000
330/1000
340/1000
350/1000
360/1000
370/1000
380/1000
390/1000
400/1000
410/1000
420/1000
430/1000
440/1000
450/1000
460/1000
470/1000
480/1000
490/1000
500/1000
510/1000
520/1000
530/1000
540/1000
550/1000
560/1000
570/1000
580/1000
590/1000
600/1000
610/1000
620/1000
630/1000
640/1000
650/1000
660/1000
670/1000
680/1000
690/1000
700/1000
710/1000
720/1000
730/1000
740/1000
750/1000
760/1000
770/1000
780/1000
790/1000
800/1000
810/1000
820/1000
830/1000
840/1000
850/1000
860/1000
870/1000
880/1000
890/1000
900/1000
910/1000
920/1000
930/1000
940/1000
950/1000
960/1000
970/1000
980/1000
990/1000


In [27]:
# Trying it out with min-max scaling
interaction_matrix_scaled = (interaction_matrix - interaction_matrix.min()) / (interaction_matrix.max() - interaction_matrix.min())

W, H = nmf_optimized(interaction_matrix_scaled, 1024, max_iter=666, tol=1e-2, debug=True)
prediction_matrix_scaled = pd.DataFrame(np.dot(W, H))

prediction_matrix_scaled.to_csv('./resources/scaled-cache.csv')

0/666


KeyboardInterrupt: 

In [28]:
# reload normalized prediction matrices

prediction_matrix_norm = pd.read_csv('./resources/norm-cache.csv', delimiter=',', index_col=0)
prediction_matrix_scaled = pd.read_csv('./resources/scaled-cache.csv', delimiter=',', index_col=0)

In [29]:
# Optimal thresholds for normalized and scaled
stats_norm = find_optimal_threshold(prediction_matrix_norm, val_df)
stats_scaled = find_optimal_threshold(prediction_matrix_scaled, val_df)

found optimal threshold 0.0098431792684458 with F1 0.1846553966189857
found optimal threshold 0.0001418035621525 with F1 0.1752298539751217


In [65]:
def make_predictions(prediction_matrix, test_set, threshold):
    
    for index, row in test_set.iterrows():
        # min handles indexing mismatch
        pred_value = prediction_matrix.iloc[min(row['user_from_id'], 3716), row['user_to_id']]
        pred = pred_value > threshold
        test_set.at[index, 'is_like'] = pred
    

No significant improvements from using normalized or min-max scaled matrix. So our best prediction-matrix was the 2048 component one with the optimal threshold found by the function.

We will now use it to make our final predictions for the test set.

In [63]:
final_threshold = stats_2048[0]
final_predictions = prediction_matrix_2048

In [66]:
make_predictions(final_predictions, nnm_test, final_threshold)

nnm_test

,user_from_id,user_to_id,is_like,is_match
0,2644,2595,False,?
1,567,2412,False,?
2,2732,3187,False,?
3,783,854,False,?
4,1104,2723,False,?
...,...,...,...,...
16198,2197,1449,False,?
16199,2507,316,False,?
16200,511,889,False,?
16201,2148,2947,False,?


Note / thoughts:
- Our predictions are quite "bad", with a best f1 score of around 0.3. We think this is due to the sparse data, low like ratio, and limitations of nmf (Changing the threshold would either increase FN or FP, big overlap here and a very small difference can cause big changes). You would get fairly bad dating recommendations
- Runtime is also extremely long, upwards of an hour to run nmf_optimized. This resulted in a lot of difficulties during the project and stopped us from trying higher component numbers.
- I could not find any mistakes (or changes) that would improve either of these (I tried)
- F1 scores have lowered across the board after rerunning things today, we think this is due to something going wrong with our caching of the prediction matrices

## Imports

In [18]:
import scipy
import numpy as np
import pandas as pd
import sys



## Data Loading

In [20]:
train_data = pd.read_csv('resources-lab-2/lab2_train.csv')
test_data = pd.read_csv('resources-lab-2/lab2_test.csv')

print(train_data)
print(test_data)

       user_from_id  user_to_id  is_like is_match
0              1136        3141    False    False
1              2424        3174    False    False
2              1300        3590    False    False
3               800        2736    False    False
4               883         437    False    False
...             ...         ...      ...      ...
76387          2376        3057    False    False
76388          1163         933    False    False
76389          2770        3324    False    False
76390           879         785    False    False
76391           291         470    False    False

[76392 rows x 4 columns]
       user_from_id  user_to_id is_like is_match
0              2644        2595       ?        ?
1               567        2412       ?        ?
2              2732        3187       ?        ?
3               783         854       ?        ?
4              1104        2723       ?        ?
...             ...         ...     ...      ...
16198          2197        1449

Okay let's see how it performs

First, we want to format our data so that we can use the method that we have already implemented in WebLab. I have decided to keep the user's ID instead of relying solely on the index to ensure accuracy.


In [32]:
from pandas import DataFrame

def compute_list_tuple_of_ids_part2(data: DataFrame):
    likes_dict = {}
    user_ids = set()  # Create a set to store unique user IDs

    for index, row in data[data['is_like']].iterrows():
        user_from_id = row['user_from_id']
        user_to_id = row['user_to_id']
        
        likes_dict.setdefault(user_from_id, set()).add(user_to_id)
        user_ids.add(user_from_id)  # Add user_from_id to the set
        user_ids.add(user_to_id)  # Add user_to_id to the set
    
    max_user_id = max(user_ids)  # Determine the maximum user ID
    
    list_of_like_tuples = [(i, likes_dict.get(i, set())) for i in range(1, max_user_id + 1)]

    return list_of_like_tuples


Also, we need to have hash functions. I decided to use the implementation from weblab for that.


In [23]:
import random

random.seed(42)

class HashFunction:
    """
    This HashFunction class can be used to create an unique hash given an alpha and beta.
    """
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta

    def hashf(self, x, n):
        """
        Returns a hash given integers x and n.
        :param x: The value to be hashed
        :param n: The number of unique ids of all sets
        :return: The hashed value x given alpha and beta
        """
        hash_value = (self.alpha * x + self.beta) % n
        return hash_value

def create_hash_functions_part2(k, max_alpha_beta):
    """
    Creates a list of k HashFunction instances with random alpha and beta values.
    :param k: The number of hash functions to create
    :param max_alpha_beta: The maximum value for alpha and beta
    :return: A list of HashFunction instances
    """
    hash_functions = []
    for _ in range(k):
        alpha = random.randint(1, max_alpha_beta)  # Ensure alpha is not 0
        beta = random.randint(0, max_alpha_beta)
        hash_function = HashFunction(alpha, beta)
        hash_functions.append(hash_function)
        return hash_functions

Now we want to compute the signature matrix. For that, I will use a slightly modified method that I implemented in Weblab. The input is a set of tuples with the ID, and the output will have the first entry in each row as the ID of the user.


In [28]:
from typing import List, Tuple

def compute_signature_part2(hashes: List[HashFunction], ids: List[Tuple[int, set]], n: int) -> np.ndarray:
    """
    This function calculates the MinHash signature matrix from the list of tuples (id, set()).
    The signature vector starts with the corresponding ID.
    :param hashes: The list of hash functions of arbitrary length
    :param ids: The list of tuples (id, set())
    :return: The MinHash signature matrix
    """
    num_rows = len(ids)
    num_cols = len(hashes)

    signature_matrix = np.full((num_rows, num_cols + 1), sys.maxsize)

    for i, (id_val, set_val) in enumerate(ids):
        signature_matrix[i, 0] = id_val  # Set the ID as the first entry in each row
        for j, hashF in enumerate(hashes):
            for element in set_val:
                hash_value = hashF.hashf(element, n)
                signature_matrix[i, j + 1] = min(signature_matrix[i,j + 1], hash_value)

    return signature_matrix

Now we want to find similar users. To achieve this, I am using Jaccard similarity with thresholds instead of top-k. I believe this approach is more flexible.


In [29]:
def find_users_with_simillar_signature_part2(user_id, signature_matrix, treshold):
    users_with_same_signature = []
    
    signature = None

    for row in signature_matrix:
        if(row[0] == user_id):
            signature = row[1:]
    if signature is None:
        return []
        
    for row in signature_matrix:
        if jacard_similarity_part2(signature, row[1:]) > treshold:
            users_with_same_signature.append(row[0])
    
    return users_with_same_signature

def jacard_similarity_part2(v1, v2):
    intersection = len(set(v1).intersection(set(v2)))
    union = len(set(v1).union(set(v2)))
    return intersection / union

def find_tuples_with_user_part2(user, lists):
    matching_tuples = []
    for item in lists:
        if item[0] == user:
            matching_tuples.append(item[1])
    return matching_tuples

def users_likes_part2(users: List[int], list_of_tuples) -> set:
    res = set()
    for user in users:
        likes = find_tuples_with_user_part2(user, list_of_tuples)
        for item in likes:
            res = res.union(item)
        
    return res

def prediction_for_user_part2(user: int, model: np.ndarray, list_of_tuples, threshold: int) -> set:
    users = find_users_with_simillar_signature_part2(user, model, threshold)
    return users_likes_part2(users, list_of_tuples)

## Testing the Recommender System

To evaluate the performance of our recommender system, we need to split our data into a train set and a test set. We will use a split ratio of 95% for the train set and 5% for the test set. Although this split ratio may not be ideal, we have chosen it for performance constraints.

As an example, we will test our system using 100 hash functions with alpha beta max of 5 and n (for modulo) of 10.


In [33]:
def find_users_likes_part2(user: int, list_of_tuples) -> set:
    for item in list_of_tuples:
        if item[0] == user:
            return item[1]
    return set()



train_set = train_data.sample(frac=0.95, random_state=42)  # 95% of the data for training
test_set = train_data.drop(train_set.index)  # Remaining 5% for testing

def test_model(model: np.ndarray, data: DataFrame, threshold: int, list_of_tuples) -> float:
    correct_predictions = 0
    total_predictions = 0
    wrong_predictions = 0
    total_wrong_predictions = 0
    counter = 0
    for index, row in data.iterrows():
        user_from_id = row['user_from_id']
        user_to_id = row['user_to_id']
        if row['is_like']:
            total_predictions += 1
            if user_to_id in prediction_for_user_part2(user_from_id, model, list_of_tuples, threshold):
                correct_predictions += 1
        if row['is_like'] is False:
            counter += 1
            if counter % 100 == 0:
                total_wrong_predictions += 1
                if user_to_id in prediction_for_user_part2(user_from_id, model, list_of_tuples, threshold):
                    wrong_predictions += 1

    return correct_predictions / total_predictions, wrong_predictions / total_wrong_predictions

list_of_tuples = compute_list_tuple_of_ids_part2(train_set)
model = compute_signature_part2(create_hash_functions_part2(100, 5), list_of_tuples, 10)

print(test_model(model, test_set, 0.6, list_of_tuples))



(0.7683881064162754, 0.25806451612903225)


# DO NOT RUN
The cell below takes a significant amount of time to run (approximately 60 minutes). In this cell, I tested various hyperparameters and manually selected the ones that I deemed most suitable.


In [ ]:
thresholds = [0.6, 0.7, 0.8]  # List of threshold values to test
k_values = [10, 50, 100]  # List of k values to test
alpha_values = [2, 5, 10]  # List of alpha values to test
n = [5, 100, 500]  # List of n values to test

for threshold in thresholds:
    for k in k_values:
        for alpha in alpha_values:
            for n_value in n:
                list_of_tuples = compute_list_tuple_of_ids_part2(train_set)
                model = compute_signature_part2(create_hash_functions_part2(k, alpha), list_of_tuples, n_value)
                accuracy, accuracy2 = test_model(model, test_set, threshold, list_of_tuples)
                print(f"Threshold: {threshold}, k: {k}, alpha: {alpha}, n: {n_value}, Correctly Positive: {accuracy}, not Correctly Negative: {accuracy2}")


#From this threshold = 0.8, k = 50, alpha = 5, n = 5  is the best

Threshold: 0.6, k: 10, alpha: 2, n: 5, Correctly Positive: 0.8325508607198748, Correctly Negative: 0.3870967741935484
Threshold: 0.6, k: 10, alpha: 2, n: 100, Correctly Positive: 0.4757433489827856, Correctly Negative: 0.16129032258064516
Threshold: 0.6, k: 10, alpha: 2, n: 500, Correctly Positive: 0.24726134585289514, Correctly Negative: 0.06451612903225806
Threshold: 0.6, k: 10, alpha: 5, n: 5, Correctly Positive: 0.9405320813771518, Correctly Negative: 0.6129032258064516
Threshold: 0.6, k: 10, alpha: 5, n: 100, Correctly Positive: 0.6666666666666666, Correctly Negative: 0.22580645161290322
Threshold: 0.6, k: 10, alpha: 5, n: 500, Correctly Positive: 0.25508607198748046, Correctly Negative: 0.06451612903225806
Threshold: 0.6, k: 10, alpha: 10, n: 5, Correctly Positive: 0.812206572769953, Correctly Negative: 0.3225806451612903
Threshold: 0.6, k: 10, alpha: 10, n: 100, Correctly Positive: 0.672926447574335, Correctly Negative: 0.3225806451612903
Threshold: 0.6, k: 10, alpha: 10, n: 500

Based on my analysis, I have decided to use the following hyperparameters: threshold = 0.8, k = 50, alpha = 5, and n = 50. The accuracy of correctly marked true values is approximately 80%, while the occurrence of false values marked as true is around 30%, which I consider acceptable. However, I am unsure if the testing methods I used are fully representative. The dataset is sparse, which posed a challenge in building the model. I am particularly concerned about the low value of n = 5, but due to time constraints, I was unable to further improve it.


In [36]:
def make_predictions_part2(model, test_set, threshold):
    
    for index, row in test_set.iterrows():
        # min handles indexing mismatch
        pred = (row['user_to_id'] in  prediction_for_user_part2(row['user_from_id'], model, list_of_tuples, threshold))
        test_set.at[index, 'is_like'] = pred

model = compute_signature_part2(create_hash_functions_part2(50, 5), list_of_tuples, 50)
dis_test = test_data.copy()

make_predictions_part2(model, dis_test, 0.8)
